In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import lightgbm as lgbm
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, train_test_split

In [4]:
TRAIN_FEATHER = 'tmp/taxi-train-v10-Airport'
TEST_FEATHER = 'tmp/taxi-test-v10-Airport'
SUBM_CSV = 'data/nyc-taxi/sample_submission.csv'
NSAMPLES = None
SEED = 1000

In [5]:
train_df = pd.read_feather(TRAIN_FEATHER)
test_df = pd.read_feather(TEST_FEATHER)
subm_df = pd.read_csv(SUBM_CSV, index_col='key')

In [6]:
train_df.shape, test_df.shape

((54075311, 24), (9914, 23))

# LGBM

In [12]:
lgbm_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.02,       # 0.1
    'feature_fraction': 0.7,     # 1.0
    'bagging_fraction': 0.7,     # 1.0
    'bagging_freq': 1,           # 0
    'num_threads':4,
#     'max_bin': 155,              # 255
#     'num_leaves': 31,            # 31
}

In [8]:
train = train_df    #.sample(NSAMPLES, random_state=SEED+i)
y = train.fare_amount.copy()
train = train[test_df.columns]

In [9]:
x_train, x_val, y_train, y_val = train_test_split(train, y, random_state=SEED, test_size=100_000)

In [10]:
dtrain = lgbm.Dataset(x_train, label=y_train, free_raw_data=False)
dval = lgbm.Dataset(x_val, label=y_val, free_raw_data=False)
dtrain.construct()
dval.construct()

In [ ]:
model = lgbm.train(
    params=lgbm_params,
    train_set=dtrain,
    valid_sets=dval,
    valid_names='validation',
    num_boost_round=100_000, 
    early_stopping_rounds=100,
    verbose_eval=500
)

Training until validation scores don't improve for 100 rounds.
[500]	validation's rmse: 3.53875
[1000]	validation's rmse: 3.4437
[1500]	validation's rmse: 3.40086
[2000]	validation's rmse: 3.37569
[2500]	validation's rmse: 3.35824


In [ ]:
model.predict(test_df)

In [ ]:
subm_df['fare_amount'] = y_preds
subm_df.to_csv('submissions/lgbm.csv')
subm_df.head()